# iso-recursive types

하스켈에는 아래와 같이 재귀적 데이타 타입(recursive data type)을 직접 정의할 수 있는 기능이 제공된다.
```haskell
data Nat = Z | S Nat deriving Show
```

하지만 하스켈에서 직접 재귀적 데이타 타입을 정의하는 기능이 없다고 가정하고
아래와 같은 재귀적 데이타 타입을 만들어내는 `Rec`이라는 타입 연산자
혹은 타입 생성자 하나만 주어지고 생성자

In [1]:
{-# LANGUAGE StandaloneDeriving, UndecidableInstances #-}
newtype Rec f = In (f (Rec f))
out (In x) = x

deriving instance Show (f (Rec f)) => Show (Rec f)

In [2]:
:kind Rec
:type In  -- 교과서에서는 fold
:type out -- 교과서에서는 unfold

Rec :: (* -> *) -> *

In :: forall (f :: * -> *). f (Rec f) -> Rec f

out :: forall (f :: * -> *). Rec f -> f (Rec f)

In [3]:
data NatF r = Z | S r deriving Show
type Nat = Rec NatF

In [4]:
z :: Nat
z   = In Z
s :: Nat -> Nat
s n = In (S n)

In [5]:
z         -- 0
s z       -- 1
s(s z)    -- 2
s(s(s z)) -- 3

In Z

In (S (In Z))

In (S (In (S (In Z))))

In (S (In (S (In (S (In Z))))))

In [6]:
data ListF r = Nil | Cons Int r deriving Show
type List = Rec ListF

In [7]:
nil :: List
nil = In Nil
cons :: Int -> List -> List
cons x xs = In (Cons x xs)

In [8]:
nil                           --
cons 1 nil
cons 1 (cons 2 nil)
cons 1 (cons 2 (cons 3 nil))

In Nil

In (Cons 1 (In Nil))

In (Cons 1 (In (Cons 2 (In Nil))))

In (Cons 1 (In (Cons 2 (In (Cons 3 (In Nil))))))

# Untyped lambda calculus with iso-recursive types

재귀적 타입(recursive type)으로 타입없는 람다계산법을 아래와 같은 하나의 타입을 가진 람다계산법으로 이해할 수도 있다.
편의상 이 재귀타입의 이름을 D라 부르자.

$D ~\triangleq~ \mu X.X \to X$

이 재귀적 데이타 타입은 $D = D \to D$의 성질을 만족한다. 즉 어떤 값을 그 값과 똑같은 타입을 인자로 받아 똑같은 타입을 돌려주는 함수로도 사용할 수 있다는 말이다. 일반적인 타입에는 이런 성질이 당연히 성립하지 않는다. 이를테면 `Int`와 `Int -> Int`가 같은 타입이라고 하면 말이 안되니까.

재귀적 타입이 없는 단순타입 람다계산법(SLTC)에서 타입을 추론할 수 없는 $(\lambda x.x~x)$와 같은 식도
재귀적 타입을 이용하면 타입을 추론할 수 있으며 일반적으로 모든 타입없는 람다식의 타입을 D로 추론할 수 있다.

일단 equi-recursive type으로 해보자면

$\displaystyle
\frac{\displaystyle
  \frac{\displaystyle
          x:D \vdash x~x : D
      }{ \vdash (\lambda x.x~x) : D\to D }
    }{ \vdash (\lambda x.x~x) : D }$

In [132]:
data DF r = DF (r -> r)
unDF (DF f) = f

type D = Rec DF

toD :: (D -> D) -> D
toD = In . DF
fromD :: D -> (D -> D)
fromD = unDF . out 

In [133]:
:type toD (\x -> x)
:type toD (\x -> toD (\y -> x))

toD (\x -> x) :: D

toD (\x -> toD (\y -> x)) :: D

In [134]:
d1 = toD (\x -> x)
d2 = toD (\x -> toD (\y -> x))

:type fromD d1 d2

fromD d1 d2 :: D

In [135]:
:type toD (\x -> fromD x x)

toD (\x -> fromD x x) :: D